# `polars` 学习簿

`polars` 是一个现代化、高性能的 `DataFrame` 数据处理库，专为处理大规模数据集而设计。它采用了全新的架构理念，使用 `Rust` 语言构建，通过 `PyO3` 提供 `Python` 接口，完美结合了系统级语言的优异性能和脚本语言的易用性。

不同于传统的 `pandas` 库，`polars` 从底层设计就专注于并行计算和内存效率，采用了 `Apache Arrow` 作为内存数据格式，实现了在多核处理器上的高效并行操作。其名称 `polars` 寓意着像北极星一样为数据科学家指引方向，提供高性能的数据处理解决方案。

以下是 `polars` 的核心架构与技术特点：

- **高性能计算引擎**：多线程并行、向量化(*SIMD*)执行、智能查询优化及高效内存管理
- **数据类型系统**：严格类型系统，支持空值安全处理、时间序列优化及嵌套数据类型
- **执行模式**：支持即时执行、惰性执行及自动选择的混合执行策略
- **数据读写能力**：多格式支持（CSV/Parquet 等）、大数据分块处理、云存储集成及数据库连接
- **数据处理操作**：提供表达式 API、聚合操作、多种连接方式及专业时间序列处理功能
- **数据可视化集成**：与主流可视化库无缝集成，支持直接绘图和 `Jupyter` 交互探索
- **与Python生态集成**：支持与 `pandas`、`numpy` 互操作，以及机器学习框架协同工作

`polars` 代表了数据处理库的新一代发展方向，将性能、内存效率和易用性结合。无论对数据科学家、分析师还是工程师，`polars` 都能提供高效、可靠的数据处理解决方案。这个工作簿的学习，有助于快速掌握 `polars` 的核心概念与功能，以更好地在实际项目中应用这些知识，提升数据处理效率和分析能力。

## 准备
你可以通过 pip来安装 `polars`：

```shell
pip install polars
```

然后就可以在程序中引入 `polars` 库了（一般用简短的别名`pl`替代）。

In [1]:
import polars as pl

下面所有的操作都假定你已经完成了上面两步。接下来，让我们从基础着手认识 `polars`。

## Series

`Series` 是 `polars` 中的一维数据结构，类似于 `pandas` 的 `Series` 或 `numpy` 的 `ndarray`。在一个 `Series` 中，所有元素都具有相同的数据类型。

In [2]:
s = pl.Series("ages", [25, 30, 35])
print(s)

shape: (3,)
Series: 'ages' [i64]
[
	25
	30
	35
]


在创建 `Series` 时，`polars` 会根据提供的值来推断数据类型。你也可以指定具体的数据类型来覆盖这种推断机制。

In [3]:
s1 = pl.Series("ints", [1, 2, 3, 4, 5])
s2 = pl.Series("uints", [1, 2, 3, 4, 5], dtype=pl.UInt64)
print(s1.dtype, s2.dtype)

Int64 UInt64


可以使用 `name` 属性获取数组名，`dtype` 属性则表示数组的数据类型。

In [4]:
print(f"Name: {s.name}")
print(f"Length: {len(s)}")
print(f"Data type: {s.dtype}")
print(f"Values: {s.to_list()}")

Name: ages
Length: 3
Data type: Int64
Values: [25, 30, 35]


`Series` 对象可以直接和标量数据进行运算，遵循与 `numpy` 类似的广播机制，如果需要可以先学习 `numpy` 的[相关知识](numpy.ipynb)。

下面是简单的例子：

In [5]:
print(s + 2)
print(s - 2)
print(s * 3)
print(s / 4)

shape: (3,)
Series: 'ages' [i64]
[
	27
	32
	37
]
shape: (3,)
Series: 'ages' [i64]
[
	23
	28
	33
]
shape: (3,)
Series: 'ages' [i64]
[
	75
	90
	105
]
shape: (3,)
Series: 'ages' [f64]
[
	6.25
	7.5
	8.75
]


`Series` 也支持向量间的基本运算，比如加减乘除等。

In [6]:
x = pl.Series("temp", [2, 3, 4])

In [7]:
print(s + x)
print(s - x)
print(s * x)
print(s / x)

shape: (3,)
Series: 'ages' [i64]
[
	27
	33
	39
]
shape: (3,)
Series: 'ages' [i64]
[
	23
	27
	31
]
shape: (3,)
Series: 'ages' [i64]
[
	50
	90
	140
]
shape: (3,)
Series: 'ages' [f64]
[
	12.5
	10.0
	8.75
]


## Series的特点

polars 和 pandas 在 `Series` 设计上存在一个显著的区别——`polars` 没有`index`概念，需要用 `alias` 重命名。理解这点能帮助我们从 `pandas` 的思维模式顺利切换到 `polars` 的思维模式。

在 **pandas** 中，每个 `Series` 都有一个显式的、强大的 `index`。这个 `index` 本身就是一个类似数组的对象，它可以包含标签（字符串、时间戳等），并且用于数据对齐和选择（通过 `loc`）。`Series` 的名称（`name`）是附加在 `index` 之上的一个属性。

在 **polars** 中，设计哲学完全不同：
*   **只有位置，没有显式索引**：polars 的 `Series` 从根本上认为数据就是一个**连续的内存块**。每个元素通过其**整数位置（0-based）** 来访问。它没有 Pandas 中那种可以自定义标签的 `index` 对象。
*   **名称只是一个标签**：`Series` 的 `name` 属性在 polars 中被视为一个简单的**字符串标签**，其主要目的是在将多个 `Series` 组合成 `DataFrame` 时作为**列名**。它不参与数据对齐或选择逻辑（polars 的数据对齐是基于表达式和名称的，而不是基于索引标签）。

正因为名称只是一个标签，polars 提供了 `alias` 方法来**安全、清晰地修改它**。

- `alias` 的作用：安全地重命名

-   **`alias` 返回一个新的 Series**：与 pandas 的 `s.rename("new_name")` 类似，`s.alias("new_name")` 会返回一个内容完全相同但名称被更改的新 `Series` 对象。原始 `Series` 不会被修改（符合函数式编程的不可变思想）。

#### 在实践中意味着什么？

当你想基于“标签”选择数据时，在 polars 中你需要换一种思维方式：

*   **pandas (基于标签)**: `df.loc[‘row_label’, ‘column_name’]`
*   **polars (基于表达式和过滤)**: `df.filter(pl.col(‘column_name’) == ‘value’)`。Polars 通过强大的表达式在**数据内容本身**上进行过滤和选择，而不是依赖一个外部的索引结构。

In [8]:
import pandas as pd

# ------------- Pandas -------------
s_pd = pd.Series([1, 2, 3], index=['a', 'b', 'c'], name='old_name')
print("Pandas Series:")
print(s_pd)
print("Index:", s_pd.index)
print("Name:", s_pd.name)
s_pd_renamed = s_pd.rename("new_pandas_name")
print("After rename:", s_pd_renamed.name)

# ------------- Polars -------------
s_pl = pl.Series('old_name', [1, 2, 3])
print("\nPolars Series:")
print(s_pl)
print("Name:", s_pl.name)
# 使用 alias 重命名
s_pl_renamed = s_pl.alias("new_polars_name")
print("After alias:", s_pl_renamed.name)

Pandas Series:
a    1
b    2
c    3
Name: old_name, dtype: int64
Index: Index(['a', 'b', 'c'], dtype='object')
Name: old_name
After rename: new_pandas_name

Polars Series:
shape: (3,)
Series: 'old_name' [i64]
[
	1
	2
	3
]
Name: old_name
After alias: new_polars_name


下面是为 `polars` 库的 `Series` 对象整理的常用功能表格，这个表格涵盖了 `Series` 最常用的功能，可以作为一份快速指南。

| 方法名称                 | 功能描述                                         | 示例代码                                     |
|--------------------------|--------------------------------------------------|----------------------------------------------|
| `dtype`                  | 返回 `Series` 对象中数据的类型                  | `s.dtype`                                    |
| `shape`                  | 返回 `Series` 对象的形状（行数）                | `s.shape`                                    |
| `name`                   | 获取或设置 `Series` 的名称                      | `s.name` 或 `s = s.rename("new_name")`       |
| `alias(name)`            | 为 `Series` 设置别名（返回新 Series）           | `s.alias("new_name")`                        |
| `head(n)`                | 返回 `Series` 对象的前 `n` 行（默认为 5）       | `s.head(3)`                                  |
| `tail(n)`                | 返回 `Series` 对象的后 `n` 行（默认为 5）       | `s.tail(2)`                                  |
| `describe()`             | 返回 `Series` 对象的统计描述                     | `s.describe()`                               |
| `is_null()`              | 返回布尔 `Series`，表示每个元素是否为 `null`     | `s.is_null()`                                |
| `is_not_null()`          | 返回布尔 `Series`，表示每个元素是否不是 `null`  | `s.is_not_null()`                            |
| `unique()`               | 返回 `Series` 中的唯一值（去重）                 | `s.unique()`                                 |
| `n_unique()`             | 返回 `Series` 中唯一值的数量                    | `s.n_unique()`                               |
| `value_counts()`        | 返回 `Series` 中每个唯一值的出现次数            | `s.value_counts()`                           |
| `sort(descending=False)` | 对 `Series` 中的元素进行排序（按值排序）         | `s.sort()` 或 `s.sort(descending=True)`      |
| `sample(n, fraction=None)` | 从 `Series` 中随机采样 n 个元素                | `s.sample(5)`                               |
| `shift(periods)`        | 将 `Series` 中的元素按指定步数进行位移          | `s.shift(1)`                                 |
| `fill_null(strategy)`   | 使用指定策略填充 `Series` 中的空值              | `s.fill_null("forward")`                     |
| `fill_nan(value)`       | 填充 `Series` 中的 NaN 值                       | `s.fill_nan(0)`                              |
| `cast(dtype)`           | 将 `Series` 转换为指定类型                       | `s.cast(pl.Float64)`                         |
| `to_list()`             | 将 `Series` 转换为 Python 列表                   | `s.to_list()`                                |
| `to_numpy()`            | 将 `Series` 转换为 numpy 数组                    | `s.to_numpy()`                               |
| `cum_sum()`             | 返回 `Series` 的累计求和                        | `s.cum_sum()`                                |
| `cum_prod()`            | 返回 `Series` 的累计乘积                        | `s.cum_prod()`                               |
| `cum_max()`             | 返回 `Series` 的累计最大值                      | `s.cum_max()`                                |
| `cum_min()`             | 返回 `Series` 的累计最小值                      | `s.cum_min()`                                |
| `rank(method)`          | 返回 `Series` 中元素的排名                      | `s.rank("average")`                          |
| `diff(n)`               | 计算 `Series` 中元素的 n 阶差分                 | `s.diff(1)`                                  |
| `pct_change(n)`         | 计算 `Series` 中元素的 n 阶百分比变化            | `s.pct_change(1)`                            |
| `clip(min, max)`        | 将 `Series` 中的值限制在 [min, max] 范围内      | `s.clip(0, 100)`                             |
| `is_between(lower, upper)` | 返回布尔 Series，表示元素是否在范围内         | `s.is_between(0, 100)`                       |
| `is_in(other)`          | 返回布尔 Series，表示元素是否在另一个集合中     | `s.is_in([1, 2, 3])`                         |
| `filter(predicate)`     | 根据谓词条件过滤 Series                         | `s.filter(pl.col("s") > 5)`                  |
| `apply(function)`       | 将函数应用于 Series 的每个元素                  | `s.apply(lambda x: x * 2)`                   |
| `map_elements(function)` | 将函数映射到 Series 的每个元素（类型安全）      | `s.map_elements(lambda x: x * 2)`           |
| `len()`                 | 返回 Series 的长度                              | `s.len()`                                    |
| `min()`                 | 返回 Series 的最小值                            | `s.min()`                                    |
| `max()`                 | 返回 Series 的最大值                            | `s.max()`                                    |
| `mean()`                | 返回 Series 的均值                              | `s.mean()`                                   |
| `median()`              | 返回 Series 的中位数                            | `s.median()`                                 |
| `sum()`                 | 返回 Series 的总和                              | `s.sum()`                                    |
| `std()`                 | 返回 Series 的标准差                            | `s.std()`                                    |
| `var()`                 | 返回 Series 的方差                              | `s.var()`                                    |
| `quantile(quantile)`    | 返回 Series 的分位数                            | `s.quantile(0.5)`                            |
| `arg_min()`             | 返回最小值的索引位置                            | `s.arg_min()`                                |
| `arg_max()`             | 返回最大值的索引位置                            | `s.arg_max()`                                |
| `corr(other)`           | 计算与另一个 Series 的相关系数                  | `s.corr(other_series)`                       |
| `cov(other)`            | 计算与另一个 Series 的协方差                    | `s.cov(other_series)`                        |
| `dot(other)`            | 计算与另一个 Series 的点积                      | `s.dot(other_series)`                        |
| `slice(offset, length)` | 从指定位置开始截取指定长度的 Series             | `s.slice(2, 5)`                             |
| `append(other)`         | 将另一个 Series 追加到当前 Series               | `s.append(other_series)`                     |
| `is_duplicated()`       | 返回布尔 Series，表示元素是否重复               | `s.is_duplicated()`                         |
| `is_unique()`           | 返回布尔 Series，表示元素是否唯一               | `s.is_unique()`                              |
| `is_finite()`           | 返回布尔 Series，表示元素是否为有限数           | `s.is_finite()`                              |
| `is_infinite()`         | 返回布尔 Series，表示元素是否为无限数           | `s.is_infinite()`                            |
| `is_nan()`              | 返回布尔 Series，表示元素是否为 NaN            | `s.is_nan()`                                 |

请注意：
1. `polars` 的 API 设计理念与 `Pandas` 有所不同，更加函数式和表达式导向
2. 许多操作在 `polars` 中通常通过表达式（`pl.col("column_name")`）来完成，而不是直接对 `Series` 进行操作
3. `polars` 默认使用惰性求值，某些操作需要在惰性上下文中使用或通过 `.collect()` 触发计算
4. 对于空值处理，`polars` 区分 `null` (缺失值) 和 `NaN` (非数字)，有不同的处理方法



## DataFrame

`DataFrame` 是 `polars` 的核心数据结构，它是一个二维表格，类似于 `pandas` 的 `DataFrame` 或关系数据库中的表。

### 创建 DataFrame

`polars` 支持从多种数据源创建 `DataFrame`，比如列表、字典或直接读取文件。下面是基于字典创建 `DataFrame` 的例子：

In [ ]:
df = pl.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "age": [25, 30, 35, 40, 45],
    "city": ["New York", "London", "Paris", "Tokyo", "Berlin"],
    "salary": [5000, 6000, 7000, 8000, 9000]
})
print(df)

# 也支持将数据写入常见的文件格式，比如 CSV、JSON 和 Parquet 等
# df.write_csv("./output.csv")

shape: (5, 4)
┌─────────┬─────┬──────────┬────────┐
│ name    ┆ age ┆ city     ┆ salary │
│ ---     ┆ --- ┆ ---      ┆ ---    │
│ str     ┆ i64 ┆ str      ┆ i64    │
╞═════════╪═════╪══════════╪════════╡
│ Alice   ┆ 25  ┆ New York ┆ 5000   │
│ Bob     ┆ 30  ┆ London   ┆ 6000   │
│ Charlie ┆ 35  ┆ Paris    ┆ 7000   │
│ David   ┆ 40  ┆ Tokyo    ┆ 8000   │
│ Eve     ┆ 45  ┆ Berlin   ┆ 9000   │
└─────────┴─────┴──────────┴────────┘


### 查看 DataFrame  

使用 `describe()` 方法来计算数据表中所有列的各种统计信息：

In [ ]:
print(df.describe())

shape: (9, 5)
┌────────────┬───────┬──────────┬────────┬────────────┐
│ statistic  ┆ name  ┆ age      ┆ city   ┆ salary     │
│ ---        ┆ ---   ┆ ---      ┆ ---    ┆ ---        │
│ str        ┆ str   ┆ f64      ┆ str    ┆ f64        │
╞════════════╪═══════╪══════════╪════════╪════════════╡
│ count      ┆ 5     ┆ 5.0      ┆ 5      ┆ 5.0        │
│ null_count ┆ 0     ┆ 0.0      ┆ 0      ┆ 0.0        │
│ mean       ┆ null  ┆ 35.0     ┆ null   ┆ 7000.0     │
│ std        ┆ null  ┆ 7.905694 ┆ null   ┆ 1581.13883 │
│ min        ┆ Alice ┆ 25.0     ┆ Berlin ┆ 5000.0     │
│ 25%        ┆ null  ┆ 30.0     ┆ null   ┆ 6000.0     │
│ 50%        ┆ null  ┆ 35.0     ┆ null   ┆ 7000.0     │
│ 75%        ┆ null  ┆ 40.0     ┆ null   ┆ 8000.0     │
│ max        ┆ Eve   ┆ 45.0     ┆ Tokyo  ┆ 9000.0     │
└────────────┴───────┴──────────┴────────┴────────────┘


使用 `df.shape` 和 `df.columns` 属性查看 `DataFrame` 的形状和列名。

In [ ]:
# 基本属性
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns}")

Shape: (5, 4)
Columns: ['name', 'age', 'city', 'salary']


`df.schema` 返回一个包含所有列名及其对应数据类型的字典，可以用来查看数据表的结构。

In [ ]:
print(f"Schema: {df.schema}")

Schema: Schema([('name', String), ('age', Int64), ('city', String), ('salary', Int64)])


`df.dtypes` 属性返回一个仅包含所有列数据类型的列表。

In [ ]:
print(f"Data types: {df.dtypes}")

Data types: [String, Int64, String, Int64]


与 `Series` 类似，创建 `DataFrame` 时，`polars` 会自动推断其结构，可以使用 `schema` 和 `schema_overrides` 覆盖某些列的推理结果。  

使用 `schema` 时，对于不指定覆盖名称的列，需要使用 `None` 表示。

In [ ]:
df0 = pl.DataFrame(
    {
        "name": ["Alice", "Ben", "Chloe", "Daniel"],
        "age": [27, 39, 41, 43],
    },
    schema={"name": None, "age": pl.UInt8},
)

print(df0)

shape: (4, 2)
┌────────┬─────┐
│ name   ┆ age │
│ ---    ┆ --- │
│ str    ┆ u8  │
╞════════╪═════╡
│ Alice  ┆ 27  │
│ Ben    ┆ 39  │
│ Chloe  ┆ 41  │
│ Daniel ┆ 43  │
└────────┴─────┘


而 `schema_overrides` 参数会更加方便，因为它允许直接忽略不想覆盖的列：

In [ ]:
df0 = pl.DataFrame(
    {
        "name": ["Alice", "Ben", "Chloe", "Daniel"],
        "age": [27, 39, 41, 43],
    },
    schema_overrides={"age": pl.UInt8},
)

print(df0)

shape: (4, 2)
┌────────┬─────┐
│ name   ┆ age │
│ ---    ┆ --- │
│ str    ┆ u8  │
╞════════╪═════╡
│ Alice  ┆ 27  │
│ Ben    ┆ 39  │
│ Chloe  ┆ 41  │
│ Daniel ┆ 43  │
└────────┴─────┘


`head()` 方法显示 `DataFrame` 的前 `n` 行。默认情况下，会显示前 `5` 行，但也可以指定想要显示的行数。

`tail()` 方法用于返回 `DataFrame` 的最后 `n` 行，类似于 `pandas` 中的 `tail`。

`sample()` 方法用于从 `DataFrame` 中随机抽取 `n` 行，默认不重复（除非指定允许重复）。

In [ ]:
print("Head:")
print(df.head(2))
print("Tail:")
print(df.tail(2))
print("Sample:")
print(df.sample(2))

Head:
shape: (2, 4)
┌───────┬─────┬──────────┬────────┐
│ name  ┆ age ┆ city     ┆ salary │
│ ---   ┆ --- ┆ ---      ┆ ---    │
│ str   ┆ i64 ┆ str      ┆ i64    │
╞═══════╪═════╪══════════╪════════╡
│ Alice ┆ 25  ┆ New York ┆ 5000   │
│ Bob   ┆ 30  ┆ London   ┆ 6000   │
└───────┴─────┴──────────┴────────┘
Tail:
shape: (2, 4)
┌───────┬─────┬────────┬────────┐
│ name  ┆ age ┆ city   ┆ salary │
│ ---   ┆ --- ┆ ---    ┆ ---    │
│ str   ┆ i64 ┆ str    ┆ i64    │
╞═══════╪═════╪════════╪════════╡
│ David ┆ 40  ┆ Tokyo  ┆ 8000   │
│ Eve   ┆ 45  ┆ Berlin ┆ 9000   │
└───────┴─────┴────────┴────────┘
Sample:
shape: (2, 4)
┌───────┬─────┬────────┬────────┐
│ name  ┆ age ┆ city   ┆ salary │
│ ---   ┆ --- ┆ ---    ┆ ---    │
│ str   ┆ i64 ┆ str    ┆ i64    │
╞═══════╪═════╪════════╪════════╡
│ Bob   ┆ 30  ┆ London ┆ 6000   │
│ David ┆ 40  ┆ Tokyo  ┆ 8000   │
└───────┴─────┴────────┴────────┘


`glimpse` 是另一个用于展示数据表前几行值的方法，但其输出格式与 `head` 不同。在这里，输出中的每一行对应一个单独的列，这使得对更宽的数据表进行检查变得更加容易:

In [ ]:
print(df.glimpse(return_as_string=True))

Rows: 5
Columns: 4
$ name   <str> 'Alice', 'Bob', 'Charlie', 'David', 'Eve'
$ age    <i64> 25, 30, 35, 40, 45
$ city   <str> 'New York', 'London', 'Paris', 'Tokyo', 'Berlin'
$ salary <i64> 5000, 6000, 7000, 8000, 9000



以下是 polars DataFrame 的常用功能表格，可以作为快速参考指南。

| 方法名称 | 功能描述 | 示例代码 |
|----------|----------|----------|
| `shape` | 返回 DataFrame 的形状（行数，列数） | `df.shape` |
| `columns` | 返回 DataFrame 的列名列表 | `df.columns` |
| `dtypes` | 返回 DataFrame 各列的数据类型列表 | `df.dtypes` |
| `schema` | 返回 DataFrame 的 schema（列名和数据类型的有序字典） | `df.schema` |
| `head(n)` | 返回前 n 行数据（默认为 5） | `df.head(10)` |
| `tail(n)` | 返回后 n 行数据（默认为 5） | `df.tail(10)` |
| `describe` | 返回 DataFrame 的统计描述 | `df.describe()` |
| `select` | 选择指定的列（可同时进行表达式计算） | `df.select(pl.col("a"), pl.col("b") * 2)` |
| `with_columns` | 添加或修改列（类似于 `select` 但保留所有列） | `df.with_columns((pl.col("a") * 2).alias("a_double"))` |
| `drop` | 删除指定的列 | `df.drop("a", "b")` |
| `rename` | 重命名列 | `df.rename({"old_name": "new_name"})` |
| `filter` | 根据条件过滤行 | `df.filter(pl.col("a") > 2)` |
| `sort` | 根据列排序 | `df.sort("a", descending=True)` |
| `group_by` | 分组（返回 GroupBy 对象，可接着进行聚合操作） | `df.group_by("a").agg(pl.col("b").sum())` |
| `join` | 连接另一个 DataFrame | `df.join(other_df, on="key", how="inner")` |
| `concat` | 拼接另一个 DataFrame（垂直或水平） | `pl.concat([df1, df2])` |
| `clone` | 创建 DataFrame 的深拷贝 | `df.clone()` |
| `lazy` | 将 DataFrame 转换为 LazyFrame，用于惰性求值 | `df.lazy()` |
| `collect` | 对 LazyFrame 执行计算并返回 DataFrame | `lazy_df.collect()` |
| `write_csv` | 将 DataFrame 写入 CSV 文件 | `df.write_csv("output.csv")` |
| `write_parquet` | 将 DataFrame 写入 Parquet 文件 | `df.write_parquet("output.parquet")` |
| `write_json` | 将 DataFrame 写入 JSON 文件 | `df.write_json("output.json")` |
| `get_column` | 根据列名获取 Series | `df.get_column("a")` |
| `to_series` | 将 DataFrame 转换为 Series（如果只有一列） | `df.to_series()` |
| `to_pandas` | 将 DataFrame 转换为 Pandas DataFrame | `df.to_pandas()` |
| `to_numpy` | 将 DataFrame 转换为 NumPy 数组 | `df.to_numpy()` |
| `to_dict` | 将 DataFrame 转换为字典 | `df.to_dict()` |
| `to_records` | 将 DataFrame 转换为记录列表 | `df.to_records()` |
| `is_empty` | 检查 DataFrame 是否为空 | `df.is_empty()` |
| `null_count` | 返回每列的空值数量 | `df.null_count()` |
| `unique` | 返回指定列的唯一值（去重） | `df.unique(subset=["a"])` |
| `sample` | 从 DataFrame 中随机采样 | `df.sample(n=5, fraction=None, with_replacement=False)` |
| `fill_null` | 填充空值 | `df.fill_null(0)` |
| `fill_nan` | 填充 NaN 值（如果列是浮点类型） | `df.fill_nan(0.0)` |
| `pipe` | 链式调用用户自定义函数 | `df.pipe(lambda df: df.with_columns(pl.col("a").alias("new_col")))` |
| `explode` | 将列表类型的列展开为多行 | `df.explode("list_col")` |
| `melt` | 将 DataFrame 从宽格式转换为长格式 | `df.melt(id_vars="id", value_vars=["a", "b"])` |
| `pivot` | 将 DataFrame 从长格式转换为宽格式 | `df.pivot(index="id", columns="category", values="value")` |
| `with_row_index` | 添加行索引列 | `df.with_row_index("index")` |
| `clear` | 清空 DataFrame 中的数据，保留 schema | `df.clear()` |
| `slice` | 按位置切片选择行 | `df.slice(10, 5)` |
| `gather_every` | 每隔 n 行选择一行 | `df.gather_every(3)` |
| `fold` | 对每行应用函数进行折叠操作 | `df.fold(lambda acc, x: acc + x, 0)` |
| `row` | 按行索引获取一行数据 | `df.row(0)` |
| `rows` | 将 DataFrame 转换为行迭代器 | `df.rows()` |
| `iter_rows` | 迭代每一行（返回命名元组） | `df.iter_rows()` |
| `partition_by` | 根据列值将 DataFrame 分割为多个 | `df.partition_by("category")` |
| `upsert` | 更新或插入数据（基于键） | `df.upsert(other_df, on="key")` |
| `interpolate` | 对数值列进行插值 | `df.interpolate()` |
| `rolling` | 滚动窗口计算 | `df.rolling(index_column="date", period="1d")` |
| `str` | 访问字符串列的方法 | `df.select(pl.col("name").str.to_uppercase())` |
| `dt` | 访问日期时间列的方法 | `df.select(pl.col("date").dt.year())` |
| `arr` | 访问数组列的方法 | `df.select(pl.col("list").arr.lengths())` |

**请注意：**

1. 表达式导向：polars 的核心是表达式系统，大多数操作通过 `pl.col("column_name")` 表达式完成
2. 惰性执行：使用 `df.lazy()` 进入惰性模式，通过 `.collect()` 触发计算
3. 不可变性：大多数操作返回新的 DataFrame，原始 DataFrame 不会被修改
4. 链式调用：Polars 鼓励使用链式调用风格：
   ```python
   result = (df.lazy()
            .filter(pl.col("age") > 18)
            .group_by("category")
            .agg(pl.col("value").mean())
            .sort("category")
            .collect())
   ```



## 表达式（Expression）

表达式（*Expression*）是 `polars` 中用于数据转换和计算的一种领域语言（*DSL*），允许以简洁的声明性语法定义对数据的操作，比一般的 Python 代码更清晰。

`polars` 基于 *Expression* 语法提供高度优化的数据查询引擎，与后面介绍的执行环境（*Context*）一起，对确保 `polars` 代码的高可读性和高性能至关重要，正确理解和运用 *Expression* 和 *Context* 是使用 `polars` 的关键。

`polars` *Expression* 主要特色是用 `pl.col()` 方法来引用列名，用这样的列名组成计算表达式，而其实际数据在之后将表达式用于特定 `DataFrame` 时才会带入计算。

这种对数据计算方案的延迟表示形式提供良好的模块化，灵活而易于复用。

比如，我们可以定义一个计算人体 BMI 的 `polars` *Expression* 如下:

In [ ]:
pl.col("weight") / (pl.col("height") ** 2)

<Expr ['[(col("weight")) / (col("heigh…'] at 0x272F77C8D50>

注意，`polars` 会将我们书写的 *Expression* 编译成一种语言无关的中间表示形式（*IR*），在实际执行时才会对其进行优化和计算。

进一步的，为了便于管理和复用，可以用变量存储表达式：

In [ ]:
bmi_expr = pl.col("weight") / (pl.col("height") ** 2)
print(bmi_expr)

[(col("weight")) / (col("height").pow([dyn int: 2]))]


## 执行环境（Context）

如前所述，*Expression* 只是一个计算公式，在定义时并不会进行任何计算，实际进行计算需要一个执行环境（*Context*）。同一个 *Expression* 根据其 *Context* 不同，也会产生不同的结果。

在本节中，我们将了解 `polars` 所提供的四种最常见的 *Context*，包括：`select`、`with_column`、`filter` 和 `group_by`。

下面的一组例子都将使用下列 `DataFrame` 数据。

In [ ]:
from datetime import date

df = pl.DataFrame(
    {
        "name": ["Alice Archer", "Ben Brown", "Chloe Cooper", "Daniel Donovan"],
        "birthdate": [
            date(1997, 1, 10),
            date(1985, 2, 15),
            date(1983, 3, 22),
            date(1981, 4, 30),
        ],
        "weight": [57.9, 72.5, 53.6, 83.1],  # (kg)
        "height": [1.56, 1.77, 1.65, 1.75],  # (m)
    }
)

print(df)

shape: (4, 4)
┌────────────────┬────────────┬────────┬────────┐
│ name           ┆ birthdate  ┆ weight ┆ height │
│ ---            ┆ ---        ┆ ---    ┆ ---    │
│ str            ┆ date       ┆ f64    ┆ f64    │
╞════════════════╪════════════╪════════╪════════╡
│ Alice Archer   ┆ 1997-01-10 ┆ 57.9   ┆ 1.56   │
│ Ben Brown      ┆ 1985-02-15 ┆ 72.5   ┆ 1.77   │
│ Chloe Cooper   ┆ 1983-03-22 ┆ 53.6   ┆ 1.65   │
│ Daniel Donovan ┆ 1981-04-30 ┆ 83.1   ┆ 1.75   │
└────────────────┴────────────┴────────┴────────┘


### select

`select` 基于 *Expression* 来创建新的列并返回一个包含这些列的 `DataFrame`。这些列的创建逻辑可以非常灵活，可以是常量，可以是其他列通过 *Expression* 计算的结果，可以是在此基础上调用聚合函数的结果。

In [ ]:
result = df.select(
    bmi=bmi_expr,
    avg_bmi=bmi_expr.mean(),
    ideal_max_bmi=25,
)
print(result)

shape: (4, 3)
┌───────────┬───────────┬───────────────┐
│ bmi       ┆ avg_bmi   ┆ ideal_max_bmi │
│ ---       ┆ ---       ┆ ---           │
│ f64       ┆ f64       ┆ i32           │
╞═══════════╪═══════════╪═══════════════╡
│ 23.791913 ┆ 23.438973 ┆ 25            │
│ 23.141498 ┆ 23.438973 ┆ 25            │
│ 19.687787 ┆ 23.438973 ┆ 25            │
│ 27.134694 ┆ 23.438973 ┆ 25            │
└───────────┴───────────┴───────────────┘


在 `select` 中使用的表达式必须生成长度完全相同的 `Series`，或者是一个标量值（常数）。标量值会进行类似 `numpy` 广播操作的扩展处理，以与其他序列的长度相匹配。

这种向量与标量之间运算时进行的广播操作也会在表达式中自动完成，例如下面这个表达式：

In [ ]:
result = df.select(deviation=(bmi_expr - bmi_expr.mean()) / bmi_expr.std())
print(result)

shape: (4, 1)
┌───────────┐
│ deviation │
│ ---       │
│ f64       │
╞═══════════╡
│ 0.115645  │
│ -0.097471 │
│ -1.22912  │
│ 1.210946  │
└───────────┘


### with_columns

`with_columns` 与 `select` 非常相似。这两者的主要区别在于：`with_columns` 会创建一个新的 `DataFrame` 对象，其中自动包含原 `DataFrame` 中的所有列以及根据其输入表达式生成的新列；而 `select` 生成的 `DataFrame` 仅包含由其输入表达式所选定的列。

In [ ]:
result = df.with_columns(
    bmi=bmi_expr,
    avg_bmi=bmi_expr.mean(),
    ideal_max_bmi=25,
)
print(result)

shape: (4, 7)
┌────────────────┬────────────┬────────┬────────┬───────────┬───────────┬───────────────┐
│ name           ┆ birthdate  ┆ weight ┆ height ┆ bmi       ┆ avg_bmi   ┆ ideal_max_bmi │
│ ---            ┆ ---        ┆ ---    ┆ ---    ┆ ---       ┆ ---       ┆ ---           │
│ str            ┆ date       ┆ f64    ┆ f64    ┆ f64       ┆ f64       ┆ i32           │
╞════════════════╪════════════╪════════╪════════╪═══════════╪═══════════╪═══════════════╡
│ Alice Archer   ┆ 1997-01-10 ┆ 57.9   ┆ 1.56   ┆ 23.791913 ┆ 23.438973 ┆ 25            │
│ Ben Brown      ┆ 1985-02-15 ┆ 72.5   ┆ 1.77   ┆ 23.141498 ┆ 23.438973 ┆ 25            │
│ Chloe Cooper   ┆ 1983-03-22 ┆ 53.6   ┆ 1.65   ┆ 19.687787 ┆ 23.438973 ┆ 25            │
│ Daniel Donovan ┆ 1981-04-30 ┆ 83.1   ┆ 1.75   ┆ 27.134694 ┆ 23.438973 ┆ 25            │
└────────────────┴────────────┴────────┴────────┴───────────┴───────────┴───────────────┘


### filter

`filter` 会根据一个或多个表达式来筛选数据表中的行，可以输入多个表达式，这些表达式会被视为逻辑与（AND）关系来进行行筛选。

In [ ]:
result = df.filter(
    pl.col("birthdate").is_between(date(1982, 12, 31), date(1996, 1, 1)),
    pl.col("height") > 1.7,
)
print(result)

shape: (1, 4)
┌───────────┬────────────┬────────┬────────┐
│ name      ┆ birthdate  ┆ weight ┆ height │
│ ---       ┆ ---        ┆ ---    ┆ ---    │
│ str       ┆ date       ┆ f64    ┆ f64    │
╞═══════════╪════════════╪════════╪════════╡
│ Ben Brown ┆ 1985-02-15 ┆ 72.5   ┆ 1.77   │
└───────────┴────────────┴────────┴────────┘


### group_by & aggregations

在 `group_by` 这一 *Context* 中，会根据表达式计算结果来对所有数据行进行分组，然后可以使用 `agg` 方法对每个分组应用一个或多个聚合函数来计算汇总统计信息。

下面的例子对出生年份进行计算，按照十年份来分组，然后将同组的人名聚合到一个列表里，顺便可以留意 `alias()` 方法提供的便利：

In [ ]:
result = df.group_by(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
).agg(pl.col("name"))
print(result)

shape: (2, 2)
┌────────┬─────────────────────────────────┐
│ decade ┆ name                            │
│ ---    ┆ ---                             │
│ i32    ┆ list[str]                       │
╞════════╪═════════════════════════════════╡
│ 1990   ┆ ["Alice Archer"]                │
│ 1980   ┆ ["Ben Brown", "Chloe Cooper", … │
└────────┴─────────────────────────────────┘


上面的例子中我们只指定了一个分组的列，我们当然也可以指定多个列来进行多级分组，比如下面的例子中先按出生年代分组，再按身高是否低于 1.7 米进行二次分组，仍然将同组的人名聚合成列表：

In [ ]:
result = df.group_by(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
    (pl.col("height") < 1.7).alias("short?"),
).agg(pl.col("name"))
print(result)

shape: (3, 3)
┌────────┬────────┬─────────────────────────────────┐
│ decade ┆ short? ┆ name                            │
│ ---    ┆ ---    ┆ ---                             │
│ i32    ┆ bool   ┆ list[str]                       │
╞════════╪════════╪═════════════════════════════════╡
│ 1980   ┆ false  ┆ ["Ben Brown", "Daniel Donovan"… │
│ 1980   ┆ true   ┆ ["Chloe Cooper"]                │
│ 1990   ┆ true   ┆ ["Alice Archer"]                │
└────────┴────────┴─────────────────────────────────┘


聚合表达式处理后生成的 `DataFrame` 中每个分组表达式和每个聚合表达式都会对应一个列，靠左侧是分组表达式，右侧是聚合表达式。

下面的例子里在上个例子基础上增加了更多的聚合表达式（注意 `.name.prefix()` 方法的使用）：

In [ ]:
result = df.group_by(
    (pl.col("birthdate").dt.year() // 10 * 10).alias("decade"),
    (pl.col("height") < 1.7).alias("short?"),
).agg(
    pl.col("name"),
    pl.len(),
    pl.col("height").max().alias("tallest"),
    pl.col("weight", "height").mean().name.prefix("avg_"),
)
print(result)

shape: (3, 7)
┌────────┬────────┬─────────────────────────────────┬─────┬─────────┬────────────┬────────────┐
│ decade ┆ short? ┆ name                            ┆ len ┆ tallest ┆ avg_weight ┆ avg_height │
│ ---    ┆ ---    ┆ ---                             ┆ --- ┆ ---     ┆ ---        ┆ ---        │
│ i32    ┆ bool   ┆ list[str]                       ┆ u32 ┆ f64     ┆ f64        ┆ f64        │
╞════════╪════════╪═════════════════════════════════╪═════╪═════════╪════════════╪════════════╡
│ 1990   ┆ true   ┆ ["Alice Archer"]                ┆ 1   ┆ 1.56    ┆ 57.9       ┆ 1.56       │
│ 1980   ┆ true   ┆ ["Chloe Cooper"]                ┆ 1   ┆ 1.65    ┆ 53.6       ┆ 1.65       │
│ 1980   ┆ false  ┆ ["Ben Brown", "Daniel Donovan"… ┆ 2   ┆ 1.77    ┆ 77.8       ┆ 1.76       │
└────────┴────────┴─────────────────────────────────┴─────┴─────────┴────────────┴────────────┘


## 惰性（Lazy）API

`polars` *Expression* 支持两种运行模式：主动模式（Eager API）和惰性模式（Lazy API）。

到目前为止的例子都使用了主动 API，在这种模式下，查询会立即执行。而在惰性模式中，只有在收集查询结果时才会对其进行评估。在很多情况下，将计算执行推迟到最后一刻能够带来显著的性能优势，这也是惰性 API 更受青睐的原因。

下面通过一个示例来说明这一点。

In [ ]:
df = pl.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "age": [25, 30, 35, 40, 45],
    "city": ["New York", "London", "Paris", "Tokyo", "Berlin"],
    "salary": [5000, 6000, 7000, 8000, 9000]
})

In [ ]:
df_filtered = df.filter(pl.col("age") > 30)
df_agg = df_filtered.group_by("city").agg(pl.col("salary").mean())
print(df_agg)

shape: (3, 2)
┌────────┬────────┐
│ city   ┆ salary │
│ ---    ┆ ---    │
│ str    ┆ f64    │
╞════════╪════════╡
│ Tokyo  ┆ 8000.0 │
│ Berlin ┆ 9000.0 │
│ Paris  ┆ 7000.0 │
└────────┴────────┘


在上面的代码中，我们使用主动 API 来：
- 读取数据
- 过滤年龄大于 `30` 的记录
- 按城市分组并计算平均薪资 

这每一步操作都会立即执行，并返回中间结果，这可能会造成浪费，因为前面的步骤可能会加载或处理后面用不到的数据，如果我们改为使用**惰性API**，它就会等到所有步骤都定义完毕之后再进行优化和执行，那么查询引擎就能规划更多的优化，比如：
- 谓词前置：在读取数据集时尽早应用过滤条件，这样就只读取年龄大于 `30` 的行；
- 投影前置：在读取数据集时仅选择所需的列，从而无需加载额外的列。

In [ ]:
q = (
    pl.scan_csv("assets/output.csv")  
    .filter(pl.col("age") > 30)  
    .group_by("city")  
    .agg(pl.col("salary").mean())  
)

df = q.collect()
print(df)

shape: (3, 2)
┌────────┬────────┐
│ city   ┆ salary │
│ ---    ┆ ---    │
│ str    ┆ f64    │
╞════════╪════════╡
│ Berlin ┆ 9000.0 │
│ Tokyo  ┆ 8000.0 │
│ Paris  ┆ 7000.0 │
└────────┴────────┘


简单说，就是把数据从加载到处理的过程定义在一个队列里，然后使用 `collect()` 方法来一次性批处理优化执行，这样 `polars` 就能对整个查询进行全局优化，显著减轻内存和 CPU 的负担，从而能够将更大的数据集存储在内存中并更快地进行处理，处理的数据集越大，效果越明显。

这些优化都是自动完成的，无需用户干预，是不是很棒？

一般来说，在大数据量的生产环境中应优先使用惰性 API，除非我们对中间结果感兴趣，或者正在进行探索性研究并且还不清楚最终查询结果是什么样子。

### 流式处理（Steaming）

惰性 API 的另一个优点是它允许以流式方式执行查询，与一次性处理所有数据不同，`polars` 可以分批执行查询，使得当数据集无法全部装入内存时也能处理，此外，流式引擎的性能也更出色。

要在使用流式模式，只要在代码中设置 `engine='streaming'` 参数即可，示例如下：

In [ ]:
q = (
    pl.scan_csv("assets/output.csv")  
    .filter(pl.col("age") > 30)  
    .group_by("city")  
    .agg(pl.col("salary").mean())  
)

df = q.collect(engine="streaming")

## 读写数据

`polars` 支持读取和写入常见的文件（例如 *CSV*、*JSON*、*Parquet*）、云存储（*S3*、*Azure Blob*、*BigQuery*）以及关系型数据库（例如 *PostgresSQL*、*MySQL* ）等数据源。

在下面的例子里，使用 `read_csv()` 方法读取 CSV 文件，使用 `write_csv()` 方法写入 CSV 文件，同样的方法也适用于其他格式的文件。

In [ ]:
# 读写 CSV 文件
# df.write_csv("output.csv")
df_from_csv = pl.read_csv("assets/output.csv")
print(df_from_csv)

# 读写 Parquet 文件（更高效）
# df.write_parquet("output.parquet")
df_from_parquet = pl.read_parquet("assets/output.parquet")
print(df_from_parquet)

# 读写 JSON 文件
# df.write_json("output.json")
df_from_json = pl.read_json("assets/output.json")
print(df_from_json)

shape: (5, 4)
┌─────────┬─────┬──────────┬────────┐
│ name    ┆ age ┆ city     ┆ salary │
│ ---     ┆ --- ┆ ---      ┆ ---    │
│ str     ┆ i64 ┆ str      ┆ i64    │
╞═════════╪═════╪══════════╪════════╡
│ Alice   ┆ 25  ┆ New York ┆ 5000   │
│ Bob     ┆ 30  ┆ London   ┆ 6000   │
│ Charlie ┆ 35  ┆ Paris    ┆ 7000   │
│ David   ┆ 40  ┆ Tokyo    ┆ 8000   │
│ Eve     ┆ 45  ┆ Berlin   ┆ 9000   │
└─────────┴─────┴──────────┴────────┘
shape: (3, 2)
┌────────┬────────┐
│ city   ┆ salary │
│ ---    ┆ ---    │
│ str    ┆ f64    │
╞════════╪════════╡
│ Tokyo  ┆ 8000.0 │
│ Berlin ┆ 9000.0 │
│ Paris  ┆ 7000.0 │
└────────┴────────┘
shape: (3, 2)
┌────────┬────────┐
│ city   ┆ salary │
│ ---    ┆ ---    │
│ str    ┆ f64    │
╞════════╪════════╡
│ Tokyo  ┆ 8000.0 │
│ Berlin ┆ 9000.0 │
│ Paris  ┆ 7000.0 │
└────────┴────────┘


## 与 pandas 对比

### 语法和用法

`polars` 的语法与 `pandas` 比较接近，但在某些方面有更简洁高效的设计，比如表达式（*Expression*）和惰性 API。实际上，`polars` 的很多设计灵感都来源于 `pandas`，但在性能和内存效率方面做了大量优化，而 `pandas` 的最新版本也在学习一些 `polars` 的设计理念。

### 性能

`polars` 在性能方面通常优于 `pandas`，尤其是在处理大规模数据集时。其多线程并行计算和内存效率使得它在执行复杂查询和数据转换时表现出色。

下面是简单的速度与内存占用对比：

In [ ]:
import time
import numpy as np

# 创建大型数据集
n_rows = 1_000_000
data = {
    "x": np.random.randn(n_rows),
    "y": np.random.randn(n_rows),
    "category": np.random.choice(["A", "B", "C"], n_rows)
}

# 创建 polars DataFrame
df_polars = pl.DataFrame(data)

# 创建 pandas DataFrame
df_pandas = pd.DataFrame(data)

# 测量聚合操作的时间
start_time = time.time()
result_polars = df_polars.group_by("category").agg(pl.col("x").mean())
end_time = time.time()
polars_time = end_time - start_time

start_time = time.time()
result_pandas = df_pandas.groupby("category")["x"].mean()
end_time = time.time()
pandas_time = end_time - start_time

print(f"polars 处理时间: {polars_time:.3f} 秒")
print(f"pandas 处理时间: {pandas_time:.3f} 秒")
print(f"速度提升: {pandas_time/polars_time:.1f} 倍")

polars 处理时间: 0.017 秒
pandas 处理时间: 0.057 秒
速度提升: 3.3 倍


In [ ]:
import sys

polars_memory = sys.getsizeof(df_polars)
pandas_memory = sys.getsizeof(df_pandas)

print(f"polars 内存使用: {polars_memory / 1024 / 1024:.2f} MB")
print(f"pandas 内存使用: {pandas_memory / 1024 / 1024:.2f} MB")
print(f"内存节省: {pandas_memory/polars_memory:.1f} 倍")

polars 内存使用: 0.00 MB
pandas 内存使用: 70.57 MB
内存节省: 1321431.5 倍


### 互操作

`polars` 可以很方便地与 `pandas` 进行互操作，但需要安装 `pyarrow` 库：

```shell
pip install pyarrow
```

然后就可以运行下面的例子了：

In [ ]:
# polars DataFrame → pandas DataFrame
pandas_df = df.to_pandas()
print(type(pandas_df))
print(pandas_df.head())

# vice versa: pandas DataFrame → polars DataFrame
df_from_pandas = pl.from_pandas(pandas_df)
print(type(df_from_pandas))
print(df_from_pandas.head())

<class 'pandas.core.frame.DataFrame'>
     city  salary
0  Berlin  9000.0
1   Paris  7000.0
2   Tokyo  8000.0
<class 'polars.dataframe.frame.DataFrame'>
shape: (3, 2)
┌────────┬────────┐
│ city   ┆ salary │
│ ---    ┆ ---    │
│ str    ┆ f64    │
╞════════╪════════╡
│ Berlin ┆ 9000.0 │
│ Paris  ┆ 7000.0 │
│ Tokyo  ┆ 8000.0 │
└────────┴────────┘


### 选型

`polars` 在以下场景中表现尤为出色：
- 处理大型数据集（GB 级别或更大）
- 需要最佳性能
- 内存受限的环境
- 需要多线程处理

而下面的场景可能更适合使用 `pandas`：
- 需要与丰富的 `pandas` 生态系统集成
- 使用许多专门的数据分析库
- 处理小型数据集且开发速度更重要

## 进阶应用示例

### 表达式 API

一些 `polars` 表达式 *Expression* 的示例。

In [ ]:
df = pl.DataFrame({
    "name": ["Alice", "Bob", "Charlie", "David", "Eve"],
    "age": [25, 30, 35, 40, 45],
    "city": ["New York", "London", "Paris", "Tokyo", "Berlin"],
    "salary": [5000, 6000, 7000, 8000, 9000]
})

In [ ]:
# 使用表达式进行复杂操作
result = df.select([
    pl.col("name"),
    pl.col("age"),
    pl.col("salary"),
    (pl.col("salary") / pl.col("age")).alias("salary_per_age"),
    pl.when(pl.col("age") > 30)
      .then(pl.col("salary") * 1.1)
      .otherwise(pl.col("salary"))
      .alias("adjusted_salary")
])
print(result)

shape: (5, 5)
┌─────────┬─────┬────────┬────────────────┬─────────────────┐
│ name    ┆ age ┆ salary ┆ salary_per_age ┆ adjusted_salary │
│ ---     ┆ --- ┆ ---    ┆ ---            ┆ ---             │
│ str     ┆ i64 ┆ i64    ┆ f64            ┆ f64             │
╞═════════╪═════╪════════╪════════════════╪═════════════════╡
│ Alice   ┆ 25  ┆ 5000   ┆ 200.0          ┆ 5000.0          │
│ Bob     ┆ 30  ┆ 6000   ┆ 200.0          ┆ 6000.0          │
│ Charlie ┆ 35  ┆ 7000   ┆ 200.0          ┆ 7700.0          │
│ David   ┆ 40  ┆ 8000   ┆ 200.0          ┆ 8800.0          │
│ Eve     ┆ 45  ┆ 9000   ┆ 200.0          ┆ 9900.0          │
└─────────┴─────┴────────┴────────────────┴─────────────────┘


### 时间序列处理

`polars` 还内置了一些常用的针对时间序列数据的处理方法。例如重采样、时间窗口和滑动窗口计算等操作。下面是一个简单的例子：

In [ ]:
# 创建时间序列数据
date_ranges = pl.date_range(
    start=pl.datetime(2023, 1, 1),
    end=pl.datetime(2023, 1, 10),
    interval="1d",
    eager=True
)

ts_df = pl.DataFrame({
    "date": date_ranges,
    "value": range(1, 11)
})
print(ts_df)

# 时间序列操作
ts_result = ts_df.with_columns([
    pl.col("value").diff().alias("diff"),
    pl.col("value").shift(1).alias("lag"),
    pl.col("value").rolling_mean(window_size=3).alias("rolling_mean")
])
print(ts_result)

shape: (10, 2)
┌────────────┬───────┐
│ date       ┆ value │
│ ---        ┆ ---   │
│ date       ┆ i64   │
╞════════════╪═══════╡
│ 2023-01-01 ┆ 1     │
│ 2023-01-02 ┆ 2     │
│ 2023-01-03 ┆ 3     │
│ 2023-01-04 ┆ 4     │
│ 2023-01-05 ┆ 5     │
│ 2023-01-06 ┆ 6     │
│ 2023-01-07 ┆ 7     │
│ 2023-01-08 ┆ 8     │
│ 2023-01-09 ┆ 9     │
│ 2023-01-10 ┆ 10    │
└────────────┴───────┘
shape: (10, 5)
┌────────────┬───────┬──────┬──────┬──────────────┐
│ date       ┆ value ┆ diff ┆ lag  ┆ rolling_mean │
│ ---        ┆ ---   ┆ ---  ┆ ---  ┆ ---          │
│ date       ┆ i64   ┆ i64  ┆ i64  ┆ f64          │
╞════════════╪═══════╪══════╪══════╪══════════════╡
│ 2023-01-01 ┆ 1     ┆ null ┆ null ┆ null         │
│ 2023-01-02 ┆ 2     ┆ 1    ┆ 1    ┆ null         │
│ 2023-01-03 ┆ 3     ┆ 1    ┆ 2    ┆ 2.0          │
│ 2023-01-04 ┆ 4     ┆ 1    ┆ 3    ┆ 3.0          │
│ 2023-01-05 ┆ 5     ┆ 1    ┆ 4    ┆ 4.0          │
│ 2023-01-06 ┆ 6     ┆ 1    ┆ 5    ┆ 5.0          │
│ 2023-01-07 ┆ 7     ┆ 1    ┆ 

## 总结

`polars` 是一个强大且高效的数据处理库，具有以下优势：
- 出色的性能，尤其适合处理大型数据集
- 内存效率高，支持惰性执行
- 丰富的数据操作功能
- 良好的生态系统集成

对于需要处理大规模数据的应用场景，`polars` 是一个很好的选择。对于熟悉 `pandas` 的用户，`polars` 的学习曲线相对平滑，大部分概念和操作都是相通的。

如果希望了解更多详细信息和高级用法，可以参考 `polars` 的[官方文档](https://pola.rs/)。